In [41]:
# Import necessary libraries
from sklearn.linear_model import *
from sklearn.metrics import accuracy_score,log_loss
from sklearn.linear_model import  LogisticRegression
from sklearn.model_selection import *
from sklearn.compose import *
from sklearn.svm import SVC
from sklearn.tree import *
from sklearn.model_selection import *
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import *
from sklearn.preprocessing import *
from sklearn.neighbors import *
from sklearn.svm import SVC
from sklearn.pipeline import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn.linear_model import *
import os

warnings.simplefilter('ignore')

os.chdir('D:\Datasets')

kyp = pd.read_csv('kyphosis.csv')
kyp

,Kyphosis,Age,Number,Start
0,absent,71,3,5
1,absent,158,3,14
2,present,128,4,5
3,absent,2,5,1
4,absent,1,4,15
...,...,...,...,...
76,present,157,3,13
77,absent,26,7,13
78,absent,120,2,13
79,present,42,7,6


In [42]:
# Extract the target variable "y" (Kyphosis) from the DataFrame
y = kyp['Kyphosis']

# Extract the feature variables "x" (all columns except 'Kyphosis') from the DataFrame
x = kyp.drop('Kyphosis', axis=1)


In [43]:
# Split the dataset into training and testing sets with a test size of 30% and stratified sampling based on the target variable
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=24)

# Create a Logistic Regression model
lr = LogisticRegression()

# Create a Gaussian Naive Bayes classifier
nb = GaussianNB()

# Create a Support Vector Classifier (SVC) with probability estimates enabled and a random state of 24
svm = SVC(random_state=24, probability=True)

# Create a Voting Classifier with Logistic Regression, Gaussian Naive Bayes, and Support Vector Machine as base estimators
# Use soft voting to combine the probabilities from the individual classifiers
# Assign weights to each base estimator: 0.76 for Logistic Regression, 0.76 for Gaussian Naive Bayes, and 0.8 for Support Vector Machine
voting = VotingClassifier([('LR', lr), ('NB', nb), ('SVM', svm)],
                          voting='soft', weights=[0.76, 0.76, 0.8])

# Fit the Voting Classifier to the training data
voting.fit(x_train, y_train)

# Use the fitted Voting Classifier to predict the classes of the test data
y_pred = voting.predict(x_test)

# Calculate and print the accuracy score of the predictions
print("Accuracy Score:", accuracy_score(y_test, y_pred))

# Get the predicted probabilities of the positive class (1) for the test data
y_pred_proba = voting.predict_proba(x_test)[:, 1]

# Calculate and print the log loss of the predicted probabilities
print("Log Loss:", log_loss(y_test, y_pred_proba))

0.76
0.4837084460839307


# gridsearch

In [44]:
# Create a Stratified K-Fold cross-validation object with 5 splits, shuffling the data, and a random state of 24
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)

# Define the parameter grid for hyperparameter tuning
params = {
    'LR__penalty': ['L2', None],
    'LR__C': np.linspace(0.001, 10, 5),
    'LR__solver': ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag'],
    'SVM__kernel': ['rbf', 'linear'],
    'SVM__C': np.linspace(0.01, 4, 5),
    'SVM__gamma': np.linspace(0.01, 4, 5)
}

# Create a Voting Classifier with Logistic Regression, Gaussian Naive Bayes, and Support Vector Machine as base estimators
# Use soft voting to combine the probabilities from the individual classifiers
voting = VotingClassifier([('LR', lr), ('NB', nb), ('SVM', svm)],
                          voting='soft')

# Create a GridSearchCV object to perform cross-validation and hyperparameter tuning
# Use the Voting Classifier as the estimator, the parameter grid defined above, the Stratified K-Fold cross-validation,
# set the verbosity to 3, and use the negative log loss as the scoring metric
gcv = GridSearchCV(voting, param_grid=params, cv=kfold, verbose=3, scoring='neg_log_loss')

# Fit the GridSearchCV object to the entire dataset (x and y)
gcv.fit(x, y)

# Print the best combination of hyperparameters found during the grid search
print("Best Parameters:", gcv.best_params_)

# Print the best score (i.e., the lowest negative log loss) found during the grid search
print("Best Score:", gcv.best_score_)

Fitting 5 folds for each of 2000 candidates, totalling 10000 fits
[CV 1/5] END LR__C=0.001, LR__penalty=L2, LR__solver=lbfgs, SVM__C=0.01, SVM__gamma=0.01, SVM__kernel=rbf;, score=nan total time=   0.0s
[CV 2/5] END LR__C=0.001, LR__penalty=L2, LR__solver=lbfgs, SVM__C=0.01, SVM__gamma=0.01, SVM__kernel=rbf;, score=nan total time=   0.0s
[CV 3/5] END LR__C=0.001, LR__penalty=L2, LR__solver=lbfgs, SVM__C=0.01, SVM__gamma=0.01, SVM__kernel=rbf;, score=nan total time=   0.0s
[CV 4/5] END LR__C=0.001, LR__penalty=L2, LR__solver=lbfgs, SVM__C=0.01, SVM__gamma=0.01, SVM__kernel=rbf;, score=nan total time=   0.0s
[CV 5/5] END LR__C=0.001, LR__penalty=L2, LR__solver=lbfgs, SVM__C=0.01, SVM__gamma=0.01, SVM__kernel=rbf;, score=nan total time=   0.0s
[CV 1/5] END LR__C=0.001, LR__penalty=L2, LR__solver=lbfgs, SVM__C=0.01, SVM__gamma=0.01, SVM__kernel=linear;, score=nan total time=   0.0s
[CV 2/5] END LR__C=0.001, LR__penalty=L2, LR__solver=lbfgs, SVM__C=0.01, SVM__gamma=0.01, SVM__kernel=linear;